# 🔍 Cómo Evaluar Modelos de Machine Learning
## Saber si un Modelo es Correcto y sus Predicciones son Confiables

---

## 🏗️ Configuración Inicial

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (mean_squared_error, r2_score, accuracy_score,
                             confusion_matrix, classification_report, roc_curve, auc)
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import seaborn as sns

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Librerías cargadas correctamente")

---

## 1. 🎯 INTRODUCCIÓN: ¿CÓMO SABEMOS SI CONFIAMOS EN EL MODELO?

### 🤔 La Pregunta Clave:

**"¿Cómo sabemos que podemos confiar en las predicciones del modelo?"**

Imagina que eres un estudiante:

- ¿Aprobó porque estudió mucho? → **BUENA RAZÓN** ✅
- ¿Aprobó por suerte? → **MALA RAZÓN** ❌

Queremos que nuestro modelo **"apruebe por buenas razones"** - es decir, que haya aprendido patrones reales, no coincidencias.

En este notebook aprenderás cómo evaluar si tu modelo es confiable usando **métricas y visualizaciones**.

---

## 2. 📊 MÉTRICAS PARA REGRESIÓN (Predecir Números)

### Preparar datos de ejemplo

In [ ]:
# Crear datos de ejemplo: predicción de precios de casas

np.random.seed(42)
tamaños = np.random.normal(150, 50, 100)
precios = 2000 * tamaños + np.random.normal(0, 30000, 100) + 50000

datos = pd.DataFrame({'tamaño': tamaños, 'precio': precios})

# Dividir en entrenamiento y prueba
X = datos[['tamaño']]
y = datos['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar modelo de regresión
modelo_reg = LinearRegression()
modelo_reg.fit(X_train, y_train)

# Hacer predicciones
y_pred = modelo_reg.predict(X_test)

print("✅ Modelo de regresión entrenado")

### 🔍 Métricas de Evaluación para Regresión

In [ ]:
# CALCULAR MÉTRICAS

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n" + "="*60)
print("📊 EVALUACIÓN DEL MODELO DE REGRESIÓN")
print("="*60)
print(f"Error Cuadrático Medio (MSE):      ${mse:,.0f}")
print(f"Raíz del Error Cuadrático (RMSE):  ${rmse:,.0f}")
print(f"Coeficiente de Determinación (R²): {r2:.3f}")
print("="*60)

In [ ]:
# VISUALIZACIONES

plt.figure(figsize=(16, 5))

# Gráfico 1: Predicciones vs Reales
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred, alpha=0.6, s=80, edgecolors='black', linewidth=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=3, label='Predicción Perfecta')
plt.xlabel('Precios Reales ($)', fontsize=11, fontweight='bold')
plt.ylabel('Precios Predichos ($)', fontsize=11, fontweight='bold')
plt.title('Predicciones vs Reales\n(Línea roja = predicción perfecta)', fontsize=12, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Gráfico 2: Distribución de Errores
plt.subplot(1, 3, 2)
errores = y_test.values - y_pred
plt.hist(errores, bins=15, edgecolor='black', alpha=0.7, color='skyblue')
plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Error = 0')
plt.xlabel('Error de Predicción ($)', fontsize=11, fontweight='bold')
plt.ylabel('Frecuencia', fontsize=11, fontweight='bold')
plt.title('Distribución de Errores\n(Debe ser simétrica alrededor de 0)', fontsize=12, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3, axis='y')

# Gráfico 3: Residuales
plt.subplot(1, 3, 3)
plt.scatter(y_pred, errores, alpha=0.6, s=80, edgecolors='black', linewidth=0.5)
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.xlabel('Predicciones ($)', fontsize=11, fontweight='bold')
plt.ylabel('Residuales/Error ($)', fontsize=11, fontweight='bold')
plt.title('Residuales vs Predicciones\n(Patrón aleatorio = BUENO)', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 📝 EXPLICACIÓN SENCILLA DE LAS MÉTRICAS

#### 1️⃣ Error Cuadrático Medio (MSE)

**¿Qué mide?** El promedio de los errores al cuadrado

**Interpretación:**
- MSE = $10,000 → El error promedio es de $100 por predicción (√10,000)
- MSE = $1,000,000 → El error promedio es de $1,000 por predicción

**¿Por qué al cuadrado?** Para penalizar más los errores grandes

---

#### 2️⃣ Coeficiente R² (Coeficiente de Determinación)

**¿Qué mide?** Qué porcentaje de la variación en el precio es explicado por el modelo

**Interpretación:**
- R² = 0.90 → El modelo explica el **90%** de la variación ✅ EXCELENTE
- R² = 0.50 → El modelo explica solo el **50%** ⚠️ REGULAR
- R² = 0.00 → El modelo no explica nada (es inútil) ❌
- R² < 0 → El modelo es peor que la media ❌❌

**Regla general:**
- R² > 0.8 → Excelente
- R² > 0.6 → Bueno
- R² > 0.4 → Aceptable
- R² < 0.4 → No confiable

---

#### 3️⃣ Patrón de Residuales

**¿Qué buscamos?** Que los errores sean **completamente aleatorios**

**BUENO ✅:**
- Puntos dispersos sin patrón claro
- Distribuidos uniformemente alrededor del cero

**MALO ❌:**
- Los errores forman una **curva** (el modelo no capta algo importante)
- Los errores grandes y pequeños están en zonas específicas
- Hay tendencia o patrón visible

---

## 3. 🎭 MÉTRICAS PARA CLASIFICACIÓN (Predecir Categorías)

### Preparar datos de ejemplo

In [ ]:
# Crear datos para clasificación: predicción de aprobación

np.random.seed(42)
horas_estudio = np.random.normal(5, 2, 200)
probabilidad_aprobacion = 1 / (1 + np.exp(-(horas_estudio - 5)))
aprobado = np.random.binomial(1, probabilidad_aprobacion)

datos_clas = pd.DataFrame({
    'horas_estudio': horas_estudio,
    'aprobo': aprobado
})

# Dividir en entrenamiento y prueba
X_clas = datos_clas[['horas_estudio']]
y_clas = datos_clas['aprobo']
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_clas, y_clas, test_size=0.3, random_state=42
)

# Entrenar modelo de clasificación
modelo_clas = LogisticRegression()
modelo_clas.fit(X_train_c, y_train_c)

# Predicciones
y_pred_c = modelo_clas.predict(X_test_c)
y_prob_c = modelo_clas.predict_proba(X_test_c)[:, 1]

print("✅ Modelo de clasificación entrenado")

### 🔍 Métricas de Evaluación para Clasificación

In [ ]:
# CALCULAR MÉTRICAS PRINCIPALES

accuracy = accuracy_score(y_test_c, y_pred_c)
cm = confusion_matrix(y_test_c, y_pred_c)

print("\n" + "="*60)
print("📊 EVALUACIÓN DEL MODELO DE CLASIFICACIÓN")
print("="*60)
print(f"Exactitud (Accuracy): {accuracy:.3f} ({accuracy*100:.1f}%)")
print("="*60)

In [ ]:
# VISUALIZACIONES

plt.figure(figsize=(16, 5))

# Gráfico 1: Matriz de Confusión
plt.subplot(1, 3, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['No Aprobó', 'Sí Aprobó'],
            yticklabels=['Real: No', 'Real: Sí'],
            annot_kws={"size": 14, "weight": "bold"})
plt.title('Matriz de Confusión', fontsize=12, fontweight='bold')
plt.ylabel('Valor Real', fontsize=11, fontweight='bold')
plt.xlabel('Predicción', fontsize=11, fontweight='bold')

# Gráfico 2: Curva ROC
plt.subplot(1, 3, 2)
fpr, tpr, thresholds = roc_curve(y_test_c, y_prob_c)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, color='darkorange', lw=3, label=f'ROC Curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Aleatorio (AUC = 0.5)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos', fontsize=11, fontweight='bold')
plt.ylabel('Tasa de Verdaderos Positivos', fontsize=11, fontweight='bold')
plt.title('Curva ROC', fontsize=12, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, alpha=0.3)

# Gráfico 3: Distribución de Probabilidades
plt.subplot(1, 3, 3)
for clase in [0, 1]:
    mask = y_test_c.values == clase
    plt.hist(y_prob_c[mask], bins=20, alpha=0.6,
            label=f'Clase {clase}', density=True, edgecolor='black')
plt.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Umbral 0.5')
plt.xlabel('Probabilidad Predicha', fontsize=11, fontweight='bold')
plt.ylabel('Densidad', fontsize=11, fontweight='bold')
plt.title('Distribución de Probabilidades\n(Bien separadas = BUENO)', fontsize=12, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# REPORTE DETALLADO DE CLASIFICACIÓN

print("\n" + "="*60)
print("📋 REPORTE DETALLADO DE CLASIFICACIÓN")
print("="*60)
print(classification_report(y_test_c, y_pred_c,
                          target_names=['No Aprobó', 'Sí Aprobó']))
print("="*60)

### 📝 EXPLICACIÓN SENCILLA DE LAS MÉTRICAS

#### 1️⃣ Exactitud (Accuracy)

**¿Qué mide?** Porcentaje de predicciones correctas

**Ejemplo:**
- Exactitud = 85% → De 100 predicciones, **85 son correctas**

**⚠️ CUIDADO:**
- Si el 90% de los estudiantes aprueban, puedo obtener 90% de exactitud **solo prediciendo siempre "aprobado"**
- Por eso la exactitud NO es suficiente cuando hay desbalance de clases

---

#### 2️⃣ Matriz de Confusión

**4 casos posibles:**

| Realidad | Predicción | Resultado | Símbolo |
|:---|:---|:---|:---:|
| Positivo | Positivo | Verdadero Positivo (TP) | ✅ |
| Negativo | Negativo | Verdadero Negativo (TN) | ✅ |
| Positivo | Negativo | Falso Negativo (FN) | ❌ |
| Negativo | Positivo | Falso Positivo (FP) | ❌ |

**Métricas derivadas:**
- **Precisión** = TP / (TP + FP) → De las que predije "sí", ¿cuántas fueron correctas?
- **Recall** = TP / (TP + FN) → De las que realmente eran "sí", ¿cuántas acerté?

---

#### 3️⃣ Curva ROC y AUC

**¿Qué es ROC?** Muestra el trade-off entre:
- **Sensibilidad** (eje Y): Detectar verdaderos positivos
- **Especificidad** (eje X): Evitar falsos positivos

**¿Qué es AUC?** Área bajo la curva ROC

**Interpretación:**
- AUC = 0.95 → Excelente (casi perfecto) ✅✅✅
- AUC = 0.85 → Muy bueno ✅✅
- AUC = 0.70 → Aceptable ✅
- AUC = 0.55 → Poco confiable ⚠️
- AUC = 0.50 → No mejor que adivinar al azar ❌

---

## 4. 🧪 PRUEBAS DE SANIDAD: ¿CÓMO SABER SI EL MODELO ES CONFIABLE?

In [ ]:
def evaluar_confiabilidad_modelo(modelo, X_test, y_test, tipo='regresion'):
    """
    Función para evaluar si podemos confiar en un modelo
    
    Parámetros:
    - modelo: modelo entrenado
    - X_test: características de prueba
    - y_test: valores reales de prueba
    - tipo: 'regresion' o 'clasificacion'
    """
    print("\n🔍 EVALUANDO CONFIABILIDAD DEL MODELO...")
    print("=" * 60)
    
    if tipo == 'regresion':
        # Predicciones
        y_pred = modelo.predict(X_test)

        # Métricas clave
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

        print(f"📊 MÉTRICAS PRINCIPALES:")
        print(f"   R²: {r2:.3f} (explica el {r2*100:.1f}% de la variación)")
        print(f"   RMSE: ${rmse:,.0f} (error promedio)")

        # Interpretación
        print(f"\n🎯 VEREDICTO:")
        if r2 > 0.8:
            print("   ✅ EXCELENTE: El modelo explica más del 80% de la variación")
        elif r2 > 0.6:
            print("   ✅ BUENO: El modelo explica más del 60% de la variación")
        elif r2 > 0.4:
            print("   ⚠️  REGULAR: El modelo necesita mejora")
        else:
            print("   ❌ MALO: El modelo no es confiable")

    else:  # clasificación
        y_pred = modelo.predict(X_test)
        y_prob = modelo.predict_proba(X_test)[:, 1]

        accuracy = accuracy_score(y_test, y_pred)
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        auc_score = auc(fpr, tpr)

        print(f"📊 MÉTRICAS PRINCIPALES:")
        print(f"   Exactitud: {accuracy:.3f} ({accuracy*100:.1f}% correcto)")
        print(f"   AUC: {auc_score:.3f}")

        # Interpretación
        print(f"\n🎯 VEREDICTO:")
        if auc_score > 0.9:
            print("   ✅ EXCELENTE: El modelo separa perfectamente las clases")
        elif auc_score > 0.8:
            print("   ✅ MUY BUENO: El modelo separa bien las clases")
        elif auc_score > 0.7:
            print("   ✅ ACEPTABLE: El modelo tiene capacidad predictiva")
        else:
            print("   ❌ POCO CONFIABLE: Similar a adivinar al azar")
    
    print("=" * 60)

# Probar con nuestros modelos
print("\n🧪 PROBANDO MODELOS EN NUESTROS DATOS...\n")

print("📈 MODELO DE REGRESIÓN:")
evaluar_confiabilidad_modelo(modelo_reg, X_test, y_test, 'regresion')

print("\n🎯 MODELO DE CLASIFICACIÓN:")
evaluar_confiabilidad_modelo(modelo_clas, X_test_c, y_test_c, 'clasificacion')

---

## 5. 🚨 SEÑALES DE ALERTA: CUÁNDO NO CONFIAR EN UN MODELO

In [ ]:
# SEÑALES DE ALERTA

print("\n🚨 SEÑALES DE QUE UN MODELO NO ES CONFIABLE:\n")

señales_alerta = {
    'REGRESIÓN': [
        "📉 R² muy bajo (< 0.3)",
        "📊 Errores con patrón claro (no aleatorios)",
        "📈 Overfitting: R² entrenamiento mucho mayor que R² prueba",
        "📉 Underfitting: R² muy bajo en ambos conjuntos"
    ],
    'CLASIFICACIÓN': [
        "🎯 Exactitud similar a la clase mayoritaria",
        "📊 AUC cerca de 0.5 (como adivinar)",
        "⚠️ Muchos falsos positivos o falsos negativos",
        "📈 Overfitting: Exactitud entrenamiento >> Exactitud prueba"
    ]
}

for tipo, señales in señales_alerta.items():
    print(f"\n{tipo}:")
    for señal in señales:
        print(f"  {señal}")

### 🔍 Demostración: Overfitting vs Buen Modelo

In [ ]:
# Crear modelo DEMASIADO COMPLEJO (overfitting)

modelo_complejo = Pipeline([
    ('poly', PolynomialFeatures(degree=10)),  # ¡Demasiado complejo!
    ('linear', LinearRegression())
])

modelo_complejo.fit(X_train, y_train)

# Evaluar en entrenamiento y prueba
train_score_complejo = modelo_complejo.score(X_train, y_train)
test_score_complejo = modelo_complejo.score(X_test, y_test)

# Comparar con nuestro modelo simple
train_score_simple = modelo_reg.score(X_train, y_train)
test_score_simple = modelo_reg.score(X_test, y_test)

print("\n" + "="*60)
print("📊 COMPARACIÓN: MODELO SIMPLE vs MODELO COMPLEJO")
print("="*60)
print("\n🧠 MODELO SIMPLE (LINEAL):")
print(f"   R² Entrenamiento: {train_score_simple:.4f}")
print(f"   R² Prueba:        {test_score_simple:.4f}")
print(f"   Diferencia:       {train_score_simple - test_score_simple:.4f}")
print(f"   → Generaliza BIEN ✅")

print("\n⚠️  MODELO COMPLEJO (GRADO 10):")
print(f"   R² Entrenamiento: {train_score_complejo:.4f}")
print(f"   R² Prueba:        {test_score_complejo:.4f}")
print(f"   Diferencia:       {train_score_complejo - test_score_complejo:.4f}")
if train_score_complejo - test_score_complejo > 0.1:
    print(f"   → OVERFITTING DETECTADO ❌❌")
print("="*60)

In [ ]:
# VISUALIZAR OVERFITTING

plt.figure(figsize=(14, 5))

# Preparar datos ordenados para visualizar la curva
X_plot = np.linspace(X_train.min() - 50, X_train.max() + 50, 300).reshape(-1, 1)

# Gráfico 1: Modelo Simple
plt.subplot(1, 2, 1)
plt.scatter(X_train, y_train, color='blue', alpha=0.6, label='Entrenamiento', s=50)
plt.scatter(X_test, y_test, color='red', alpha=0.6, label='Prueba', s=50)
y_plot_simple = modelo_reg.predict(X_plot)
plt.plot(X_plot, y_plot_simple, 'g-', linewidth=3, label='Predicción')
plt.xlabel('Tamaño (m²)', fontsize=11, fontweight='bold')
plt.ylabel('Precio ($)', fontsize=11, fontweight='bold')
plt.title(f'MODELO SIMPLE (R² = {test_score_simple:.3f})\nBUEN GENERALIZADOR ✅', 
          fontsize=12, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Gráfico 2: Modelo Complejo
plt.subplot(1, 2, 2)
plt.scatter(X_train, y_train, color='blue', alpha=0.6, label='Entrenamiento', s=50)
plt.scatter(X_test, y_test, color='red', alpha=0.6, label='Prueba', s=50)
y_plot_complejo = modelo_complejo.predict(X_plot)
plt.plot(X_plot, y_plot_complejo, 'orange', linewidth=3, label='Predicción')
plt.xlabel('Tamaño (m²)', fontsize=11, fontweight='bold')
plt.ylabel('Precio ($)', fontsize=11, fontweight='bold')
plt.title(f'MODELO COMPLEJO (R² = {test_score_complejo:.3f})\nOVERFITTING ❌', 
          fontsize=12, fontweight='bold', color='red')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 6. ✅ CHECKLIST: ¿PUEDO CONFIAR EN ESTE MODELO?

### 📊 PARA REGRESIÓN:

- [ ] **R² > 0.6** (explica al menos 60% de la variación)
- [ ] **Los errores son aleatorios** (sin patrón claro)
- [ ] **RMSE es aceptable** para el negocio
- [ ] **No hay overfitting** (R² entrenamiento ≈ R² prueba)

### 🎯 PARA CLASIFICACIÓN:

- [ ] **Exactitud > clase mayoritaria** (mejor que adivinar)
- [ ] **AUC > 0.7** (mejor que adivinar al azar)
- [ ] **Matriz de confusión balanceada** (buenos TP y TN)
- [ ] **Buen balance precisión-recall** según necesidad del negocio

### 🔧 GENERAL (AMBOS TIPOS):

- [ ] **Datos train y test similares** (estadísticas parecidas)
- [ ] **Modelo es estable** (mismos resultados con diferentes divisiones)
- [ ] **Predicciones tienen sentido** para el dominio del negocio
- [ ] **No hay variables que filtren información futura** (data leakage)

### 🎯 REGLA DE ORO:

> **"Si no puedes explicar por qué el modelo hace ciertas predicciones, NO deberías confiar en él"**

---

## 7. 🧪 TEST PRÁCTICO: EVALÚA TÚ MISMO

In [ ]:
def test_tu_comprension():
    """
    Ejercicio práctico para evaluar tu comprensión
    """
    print("\n🧪 TEST PRÁCTICO: EVALÚA ESTOS MODELOS\n")
    print("="*60)

    # Escenario 1
    print("\n📊 ESCENARIO 1: Modelo de Regresión")
    print("   - R² entrenamiento: 0.95")
    print("   - R² prueba: 0.55")
    print("   - Errores: muestran patrón en forma de U")
    print("\n   ¿Confiarías en este modelo? (sí/no):")
    print("   Respuesta: NO ❌")
    print("   Explicación: Hay OVERFITTING (0.95 >> 0.55)")
    print("               Los errores tienen PATRÓN claro")

    # Escenario 2
    print("\n📊 ESCENARIO 2: Modelo de Clasificación")
    print("   - Exactitud: 0.92")
    print("   - Clase mayoritaria: 90%")
    print("   - AUC: 0.65")
    print("\n   ¿Confiarías en este modelo? (sí/no):")
    print("   Respuesta: NO ❌")
    print("   Explicación: El modelo solo REPLICA la clase mayoritaria")
    print("               AUC bajo (0.65) indica poca capacidad predictiva")
    print("               La exactitud se engañosa")

    # Escenario 3
    print("\n📊 ESCENARIO 3: Modelo de Regresión")
    print("   - R² entrenamiento: 0.78")
    print("   - R² prueba: 0.75")
    print("   - Errores: distribuidos aleatoriamente")
    print("\n   ¿Confiarías en este modelo? (sí/no):")
    print("   Respuesta: SÍ ✅")
    print("   Explicación: BUENA GENERALIZACIÓN (0.78 ≈ 0.75)")
    print("               Explica el 75% de la variación")
    print("               Errores aleatorios = modelo captura patrones reales")

    print("\n" + "="*60)

# Ejecutar test
test_tu_comprension()

---

## 8. 📈 RESUMEN VISUAL: CÓMO EVALUAR MODELOS

In [ ]:
# CREAR RESUMEN VISUAL

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('RESUMEN: Cómo Evaluar Modelos', fontsize=16, fontweight='bold', y=1.00)

# REGRESIÓN - BUEN MODELO
x_bueno = np.linspace(0, 10, 100)
y_bueno = 2*x_bueno + np.random.normal(0, 1, 100)
y_pred_bueno = 2*x_bueno

axes[0, 0].scatter(x_bueno, y_bueno, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[0, 0].plot(x_bueno, y_pred_bueno, 'r-', linewidth=3)
axes[0, 0].set_title('✅ BUEN MODELO DE REGRESIÓN\n(Puntos cerca de la línea)', 
                      fontsize=12, fontweight='bold', color='green')
axes[0, 0].set_xlabel('X')
axes[0, 0].set_ylabel('Y')
axes[0, 0].grid(True, alpha=0.3)

# REGRESIÓN - MAL MODELO
x_malo = np.linspace(0, 10, 100)
y_malo = np.sin(x_malo)*3 + np.random.normal(0, 2, 100)
y_pred_malo = np.ones(100) * np.mean(y_malo)

axes[0, 1].scatter(x_malo, y_malo, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[0, 1].plot(x_malo, y_pred_malo, 'r-', linewidth=3)
axes[0, 1].set_title('❌ MAL MODELO DE REGRESIÓN\n(Puntos dispersos lejos de línea)', 
                      fontsize=12, fontweight='bold', color='red')
axes[0, 1].set_xlabel('X')
axes[0, 1].set_ylabel('Y')
axes[0, 1].grid(True, alpha=0.3)

# CLASIFICACIÓN - BUEN MODELO
x_clas_bueno = np.random.normal(0, 1, 100)
y_clas_bueno = (x_clas_bueno > 0).astype(int) + np.random.normal(0, 0.3, 100)

scatter1 = axes[1, 0].scatter(x_clas_bueno, y_clas_bueno, 
                               c=(y_clas_bueno>0.5).astype(int), cmap='RdYlGn', 
                               alpha=0.7, s=80, edgecolors='black', linewidth=0.5)
axes[1, 0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_title('✅ BUEN MODELO DE CLASIFICACIÓN\n(Clases bien separadas)', 
                      fontsize=12, fontweight='bold', color='green')
axes[1, 0].set_xlabel('X')
axes[1, 0].set_ylabel('Y')
axes[1, 0].grid(True, alpha=0.3)

# CLASIFICACIÓN - MAL MODELO
x_clas_malo = np.random.normal(0, 1, 100)
y_clas_malo = np.random.randint(0, 2, 100)

scatter2 = axes[1, 1].scatter(x_clas_malo, y_clas_malo, 
                               c=y_clas_malo, cmap='RdYlGn', 
                               alpha=0.7, s=80, edgecolors='black', linewidth=0.5)
axes[1, 1].set_title('❌ MAL MODELO DE CLASIFICACIÓN\n(Clases mezcladas)', 
                      fontsize=12, fontweight='bold', color='red')
axes[1, 1].set_xlabel('X')
axes[1, 1].set_ylabel('Y')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 9. 🏆 CONCLUSIÓN FINAL

### 🎯 RESUMEN: CÓMO SABER SI UN MODELO ES CONFIABLE

#### ✅ SEÑALES DE UN BUEN MODELO:

**REGRESIÓN:**
- R² alto (> 0.7 ideal)
- Errores pequeños y aleatorios
- Predicciones cerca de la línea perfecta
- Similar rendimiento en entrenamiento y prueba

**CLASIFICACIÓN:**
- Exactitud mayor que clase mayoritaria
- AUC alto (> 0.8 ideal)
- Matriz de confusión balanceada
- Buen balance precisión-recall

---

#### 🚨 SEÑALES DE ALERTA:

- **Overfitting**: Entrenamiento >> Prueba
- **Underfitting**: Ambos rendimientos muy bajos
- **Errores con patrón**: No son aleatorios
- **Métricas engañosas**: Similar a adivinar al azar

---

#### 💡 RECUERDA SIEMPRE:

> **"Confiar en un modelo no es solo sobre números."**
> 
> **"Es sobre entender POR QUÉ hace las predicciones que hace."**
> 
> **"Un modelo perfecto en papel puede ser peligroso en la práctica si no entendemos su comportamiento."**
> 
> **"La evaluación honesta es la clave para modelos confiables."** 🗝️

---

# 🎉 ¡Has aprendido cómo evaluar modelos correctamente!

Ahora puedes:
- ✅ Interpretar métricas de regresión y clasificación
- ✅ Detectar overfitting y underfitting
- ✅ Crear visualizaciones para evaluar modelos
- ✅ Decidir si confiar en un modelo

**¡Sigue practicando y creando modelos cada vez mejores! 🚀**